In [3]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F24 '''
def F24(X):
    f = bn.F24()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.0097 ,random_state=0 , l1_ratio= 0.04, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 59.61 , C = 38.66 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_241000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_24_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [8]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=957931, Mon Apr 13 13:31:49 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 2.256806110091113e+03 1.0e+00 1.23e-01  1e-01  1e-01 0:00.0
    2     26 2.256612422144683e+03 1.1e+00 1.20e-01  1e-01  1e-01 0:00.0
    3     39 2.245028907380159e+03 1.1e+00 1.23e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2259.02145337] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2258.00539813] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 1.067640746447148e+03 1.6e+00 4.51e-03  4e-03  4e-03 0:00.9
  200   2600 1.066727240816747e+03 1.8e+00 7.59e-05  5e-05  6e-05 0:01.8
  300   3900 1.066727093031510e+03 1.9e+00 1.05e-06  6e-07  8e-07 0:02.7
  400   5200 1.066727093008921e+03 1.9e+00 1.55e-08  7e-09  1e-08 0:03.6
  455   5915 1.066727093008918e+03 2.1e+00 2.38e-09  1e-09  1e-09 0:04.1
termination on tolfun=1e-11 (Mon Apr 13 13:31:54 2020)
final/bestever f-value = 1.066727e+03 1.066727e+03
incumbent solution: [-0.25324697  0.08136035 -0.01992649 -0.08205287 -0.18363419  0.04604757
  0.20994824 -0.0728177  ...]
std deviations: [1.34181608e-09 1.13051948e-09 1.14206023e-09 1.18977730e-09
 1.15372559e-09 1.25089706e-09 1.36540804e-09 1.49824408e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=896191, Mon Apr 13 13:31:54 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 2.266281990329619e+03 1.0e+00 1.19e-01  1e-01  1e-01 0:00.0
termination on tolfun=1e-1

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2266.28199033] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=2266.281990, sigma=1.19e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=2266.281990, sigma=1.19e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

   83   1079 -4.051015648063485e+08 2.4e+00 1.19e+02  1e+02  1e+02 0:00.9
termination on tolfacupx=1000.0 (Mon Apr 13 13:31:56 2020)
final/bestever f-value = -4.001615e+08 -4.051016e+08
incumbent solution: [-517.18014455  -24.2194734  -267.19655168 -410.99917447 -463.69404169
  318.94421477  292.15127351 -382.82905582 ...]
std deviations: [127.90607892 109.38425366 113.72191144 117.76572391 112.33633134
 107.91993252 110.92104853 115.67522851 ...]
Run : 1
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=826654, Mon Apr 13 13:31:56 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 2.260227917227854e+03 1.0e+00 1.18e-01  1e-01  1e-01 0:00.0
    2     26 2.260224382485251e+03 1.1e+00 1.12e-01  1e-01  1e-01 0:00.0
    3     39 2.260204940165767e+03 1.1e+00 1.09e-01  1e-01  1e-01 0:00.0


...

   86   1118 -3.382170295434751e+08 2.9e+00 1.07e+02  9e+01  1e+02 0:00.9
termination on tolfacupx=1000.0 (Mon Apr 13 13:34:50 2020)
final/bestever f-value = -3.190566e+08 -3.382170e+08
incumbent solution: [-593.40435597  -86.52777105   70.54346662    1.64705797   30.56528833
  219.07552804  164.98954232 -347.07620004 ...]
std deviations: [114.09638545 102.92566763 101.12955722 101.34419477  98.81903443
  98.38645761  97.99883418 106.18042174 ...]
Run : 29
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=827682, Mon Apr 13 13:34:50 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 2.260219719837427e+03 1.0e+00 1.21e-01  1e-01  1e-01 0:00.0
    2     26 2.260197593119379e+03 1.1e+00 1.20e-01  1e-01  1e-01 0:00.0
    3     39 2.260079518511778e+03 1.1e+00 1.23e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2260.24040702] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2260.2325134] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 1.070392077409544e+03 1.8e+00 1.10e-02  9e-03  1e-02 0:00.9
  200   2600 1.066727825867757e+03 1.8e+00 1.86e-04  1e-04  2e-04 0:01.8
  300   3900 1.066727093281349e+03 1.8e+00 3.36e-06  2e-06  2e-06 0:02.6
  400   5200 1.066727093008973e+03 1.9e+00 5.56e-08  3e-08  4e-08 0:03.5
  485   6305 1.066727093008918e+03 2.0e+00 3.71e-09  2e-09  2e-09 0:04.3
termination on tolfun=1e-11 (Mon Apr 13 13:34:55 2020)
final/bestever f-value = 1.066727e+03 1.066727e+03
incumbent solution: [-0.25324697  0.08136036 -0.01992649 -0.08205287 -0.18363418  0.04604756
  0.20994824 -0.0728177  ...]
std deviations: [1.81642531e-09 1.88487953e-09 1.75470440e-09 1.85249352e-09
 1.76530577e-09 1.82834825e-09 1.88105292e-09 1.87752472e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=874729, Mon Apr 13 13:34:55 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 2.266281990329619e+03 1.0e+00 1.19e-01  1e-01  1e-01 0:00.0
termination on tolfun=1e-1

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2266.28199033] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=2266.281990, sigma=1.19e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=2266.281990, sigma=1.19e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

   85   1105 -4.516003607147636e+08 2.8e+00 1.30e+02  1e+02  1e+02 0:00.9
termination on tolfacupx=1000.0 (Mon Apr 13 13:34:56 2020)
final/bestever f-value = -4.128084e+08 -4.516004e+08
incumbent solution: [ 573.77601831 -211.71311076  -30.42306402 -148.20616934  399.81935807
 -488.28081094 -568.57222485  409.81637799 ...]
std deviations: [123.832439   120.62291902 117.12077363 130.47681455 126.60832658
 138.18025471 140.05026324 133.27283131 ...]


In [9]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F24_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
KNN_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F24(X_Values.iloc[i,:25])
    SVM_Fun [i] = F24(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F24(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([438.41283019]), count=array([1])) 15.689729109038616
SVM
ModeResult(mode=array([403.62113791]), count=array([1])) 34.77300075927641
ELN
ModeResult(mode=array([2.55854947e+10]), count=array([1])) 8911446839.917683
